In [6]:
import intake

cat = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")
cat

pangeo-cmip6-ESM Collection with 224674 entries:
	> 15 activity_id(s)

	> 26 institution_id(s)

	> 58 source_id(s)

	> 100 experiment_id(s)

	> 128 member_id(s)

	> 29 table_id(s)

	> 297 variable_id(s)

	> 9 grid_label(s)

	> 224674 zstore(s)

	> 60 dcpp_init_year(s)

In [14]:
cat.df.grid_label.unique()

array(['gn', 'gr', 'grz', 'gnz', 'gr1', 'gm', 'gr2', 'gr1z', 'gr3'],
      dtype=object)

In [20]:
tmp = cat.search(activity_id='CMIP', table_id='day', variable_id='tas', experiment_id='historical', grid_label='gn')
tmp

pangeo-cmip6-ESM Collection with 62 entries:
	> 1 activity_id(s)

	> 7 institution_id(s)

	> 9 source_id(s)

	> 1 experiment_id(s)

	> 33 member_id(s)

	> 1 table_id(s)

	> 1 variable_id(s)

	> 1 grid_label(s)

	> 62 zstore(s)

	> 0 dcpp_init_year(s)

In [15]:
tmp.df.grid_label.unique()

array(['gn', 'gr', 'gr1', 'gr2'], dtype=object)

In [ ]:
models = ['NESM3', 'HadGEM3-GC31-LL', 'EC-Earth3-Veg', 'MIROC6','MIROC-ES2L','GFDL-CM4',
          'UKESM1-0-LL','CAMS-CSM1-0','CESM2','SAM0-UNICON','CanESM5','IPSL-CM6A-LR','CNRM-CM6-1','CNRM-ESM2-1']

bigdict = dict()
for model in models:
    print(model)
    tmp = cat.search(source_id=model, member_id='r1i1p1f1',
                     variable_id=['thetao', 'so', 'siconc'], experiment_id='historical', table_id=['SImon', 'Omon'])
    tmpdict = tmp.to_dataset_dict(zarr_kwargs={'consolidated': True},
                                  cdf_kwargs={'chunks': {}})
    bigdict.update(tmpdict)
    tmp = None
    tmpdict = None



NESM3
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)
HadGEM3-GC31-LL
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 0 group(s)
EC-Earth3-Veg
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)
MIROC6
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
MIROC-ES2L
--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 0 group(s)
GFDL-CM4
--> The keys in t

In [3]:
model = 'SAM0-UNICON'
tmp = cat.search(source_id=model, member_id='r1i1p1f1',
           variable_id=['thetao', 'so', 'siconc'], experiment_id='historical', table_id=['SImon', 'Omon'])
tmp.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
17089,CMIP,SNU,SAM0-UNICON,historical,r1i1p1f1,Omon,so,gn,gs://cmip6/CMIP/SNU/SAM0-UNICON/historical/r1i...,NaN
17093,CMIP,SNU,SAM0-UNICON,historical,r1i1p1f1,Omon,thetao,gn,gs://cmip6/CMIP/SNU/SAM0-UNICON/historical/r1i...,NaN
17102,CMIP,SNU,SAM0-UNICON,historical,r1i1p1f1,SImon,siconc,gn,gs://cmip6/CMIP/SNU/SAM0-UNICON/historical/r1i...,NaN


In [5]:
import xarray as xr
import fsspec
dsets = [xr.open_zarr(fsspec.get_mapper(item), consolidated=True)
         for item in tmp.df.zstore.values]

In [ ]:
tmp.to_dataset_dict(zarr_kwargs={'consolidated': True})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 2 group(s)


In [6]:
dsets

[<xarray.Dataset>
 Dimensions:             (bnds: 2, i: 320, j: 384, lev: 60, time: 1980, vertices: 4)
 Coordinates:
   * i                   (i) int32 0 1 2 3 4 5 6 ... 313 314 315 316 317 318 319
   * j                   (j) int32 0 1 2 3 4 5 6 ... 377 378 379 380 381 382 383
     latitude            (j, i) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   * lev                 (lev) float64 5.0 15.0 25.0 ... 5.125e+03 5.375e+03
     lev_bnds            (lev, bnds) float64 dask.array<chunksize=(60, 2), meta=np.ndarray>
     longitude           (j, i) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
   * time                (time) object 1850-01-17 00:30:00 ... 2014-12-16 12:00:00
     time_bnds           (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
 Dimensions without coordinates: bnds, vertices
 Data variables:
     so                  (time, lev, j, i) float32 dask.array<chunksize=(12, 60, 384, 320), meta=np.ndarray>
     vertices_latitud

In [4]:
import xarray as xr
url = 'https://data.nodc.noaa.gov/thredds/dodsC/ncei/woa/temperature/A5B7/1.00/woa18_A5B7_t00_01.nc'
ds = xr.open_dataset(url, decode_times=False)
ds

<xarray.Dataset>
Dimensions:             (depth: 102, lat: 180, lon: 360, nbounds: 2, time: 1)
Coordinates:
  * lat                 (lat) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * lon                 (lon) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * depth               (depth) float32 0.0 5.0 10.0 ... 5300.0 5400.0 5500.0
  * time                (time) float32 726.0
Dimensions without coordinates: nbounds
Data variables:
    crs                 int32 ...
    lat_bnds            (lat, nbounds) float32 ...
    lon_bnds            (lon, nbounds) float32 ...
    depth_bnds          (depth, nbounds) float32 ...
    climatology_bounds  (time, nbounds) float32 ...
    t_an                (time, depth, lat, lon) float32 ...
    t_mn                (time, depth, lat, lon) float32 ...
    t_dd                (time, depth, lat, lon) float64 ...
    t_sd                (time, depth, lat, lon) float32 ...
    t_se                (time, depth, lat, lon) float32 ...
    t_oa  

In [3]:
ds.nbytes/1e6

3807.15676